<a href="https://colab.research.google.com/github/silverstar0727/1day-1commit-challenge/blob/master/machine_learning_ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 전처리

In [0]:
# 데이터 생성
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1,2,3,4
5,6,,8
10,11,12,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1,2,3.0,4.0
1,5,6,NaN,8.0
2,10,11,12.0,NaN


In [0]:
# 누락된 데이터개수 확인
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [0]:
# 누락된 데이터가 존재하는 행 삭제
df.dropna(axis = 0)

,A,B,C,D
0,1,2,3.0,4.0


In [0]:
# 누락된 데이터가 존재하는 열 삭제
df.dropna(axis = 1)

,A,B
0,1,2
1,5,6
2,10,11


In [0]:
# 모든 요소가 누락되었을때 삭제
df.dropna(how = 'all')

,A,B,C,D
0,1,2,3.0,4.0
1,5,6,NaN,8.0
2,10,11,12.0,NaN


In [0]:
# 특정열에 누락된 값이 잇는 "행" 삭제
df.dropna(subset = ['C'])

,A,B,C,D
0,1,2,3.0,4.0
2,10,11,12.0,NaN


In [0]:
# 누락된 값 채우기
import numpy as np
from sklearn.impute import SimpleImputer

simr = SimpleImputer(missing_values = np.nan, strategy = 'mean')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [0]:
# 행방향으로 바꾸기 (전치후 위과정을 거치고 다시 전치))
from sklearn.preprocessing import FunctionTransformer

ftr_simr = FunctionTransformer(lambda X: simr.fit_transform(X.T).T, validate = False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

## 범주형 데이터 다루기 - 매핑


In [0]:
import pandas as pd

df = pd.DataFrame([['green','M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


#### 순서매핑


In [0]:
#순서 매핑

size_mapping = {
    'XL':3,
    'L':2,
    'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [0]:
# 거꾸로 매핑하기
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

#### 클래스 매핑

In [0]:
# class mapping
import numpy as np
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [0]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [0]:
# 매핑 딕셔너리의 키-값 쌍을 뒤집어서 다시한번 원본으로 맵핑
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [0]:
# 사이킷런으로 매핑구현
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [0]:
# 사이킷런 재변환
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

## 데이터셋 나누기


In [0]:
# wine dataset 사용, 13개의 특성, 178개의 요소로 구성

df_wine = pd.read_csv('https://archive.ics.uci.edu/''ml/machine-learning-databases/''wine/wine.data', header = None)

df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Flavonoids',
                   'Total phenols', 'Nonflavanoid  phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                   '0D280/OD315 of diluted wines', 'Proline']
        
print( np.unique(df_wine['Class label']))

[1 2 3]


In [0]:
df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Flavonoids,Total phenols,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,0D280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [0]:
#사이킷런을 이용한 데이터 분할
from sklearn.model_selection import train_test_split

X,y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
X_train, X_test, y_train, y_test = \
train_test_split(X,y, test_size = 0.3, random_state = 0, stratify = y)

## 원-핫 인코딩(순서없는 특성)

In [0]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X


array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ohe = OneHotEncoder(categories = 'auto')
col_trans = ColumnTransformer([('ohe', ohe, [0])],remainder = 'passthrough')
col_trans.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [0]:
# pandas로 원핫 인코딩 더미변수 만들기
import pandas as pd

pd.get_dummies(df[['price', 'color', 'size']])


,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0
